<a href="https://colab.research.google.com/github/Kira1108/huggingface-examples/blob/main/CustomDatasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### When you are working on ML projects, you are expected to write dirty code.     
- Don't use design pattern at all.    
- Don't use data structure algorithms at all.   
- Use vectorized operations.    
- Use encapsulated packages, like scikit-learn, tensorflow etc.      
- Exploring data first and carefully.    

**Install Transformer Packages**

In [1]:
# from IPython.display import clear_output

# !pip install transformers datasets

# clear_output()

**Download Raw Dataset**

In [2]:
# !wget -nc https://lazyprogrammer.me/course_files/AirlineTweets.csv
# !mv AirlineTweets.csv /data

In [3]:
import logging
logger = logging.getLogger("artifacts")

import pandas as pd
import os
import json
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from joblib import load, dump

**Clean data for transformer training**    

Althrough it is not a starndard way of transforming labels and prepare training dataset   
It is required by transformers.     

In [4]:
class ArtifactStore:
    """`ArtifactStore` stores files that you created when doint ML.
        :Param: artifacts_fq: root folder of artifacts path(default to `.artifacts`)
    """
    
    def __init__(self, artifacts_fp = "./artifacts"):
        self.artifacts_fp = Path(artifacts_fp)
        os.makedirs(self.artifacts_fp,exist_ok=True)

    def log_binary(self, obj, fname):
        fpath = self.artifacts_fp / f"{fname}.joblib"
        dump(obj,fpath)
        logger.info(f"Dumped binary to {fpath}")
        
    def load_binary(self, fname):
        fpath = self.artifacts_fp / f"{fname}.joblib"
        return load(fpath)
        
    def log_json(self, obj, fname):
        fpath = self.artifacts_fp / f"{fname}.json"
        json.dump(obj, open(fpath,'w'))
        logger.info(f"Dumped json file to {fpath}")
        
    def load_json(self, fname):
        fpath = self.artifacts_fp / f"{fname}.json"
        return json.load(open(fpath,'r'))
        
    def log_label_encoder(self, label_encoder):
        self.log_binary(label_encoder,"label_encoder")
        
        classmap = {i:c for i,c in enumerate(label_encoder.classes_)}
        self.log_json(classmap,'label_encoder_classmap')
            
alog = ArtifactStore()

**Don't try to write better code when doing ML.(Do that only if this code makes money for you)**

In [5]:
# 0. do settings
DATA_PATH = Path('./data')
ARTIFACTS_PATH = Path("./artifacts")

# 1. read data
df = pd.read_csv(DATA_PATH / "AirlineTweets.csv")[['text','airline_sentiment']]

# 2. ml preprocessing
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['airline_sentiment'])

# 3. do dirty column operations
df.drop('airline_sentiment', axis = 1, inplace = True)
df.rename(columns = {'text':'sentence'}, inplace = True)

# 4. log whatever that will be used in the future
alog.log_label_encoder(label_encoder)

# 5. additional steps for your task
df.to_csv(DATA_PATH / "train_data.csv", index = False)

# 6. validate the steps above
print(df.head(5))

                                            sentence  label
0                @VirginAmerica What @dhepburn said.      1
1  @VirginAmerica plus you've added commercials t...      2
2  @VirginAmerica I didn't today... Must mean I n...      1
3  @VirginAmerica it's really aggressive to blast...      0
4  @VirginAmerica and it's a really big bad thing...      0


**Load artifacts for later use**

In [6]:
alog.load_binary('label_encoder')\
    .transform(['negative','positive','neutral','positive'])

array([0, 2, 1, 2])

In [7]:
alog.load_json('label_encoder_classmap')

{'0': 'negative', '1': 'neutral', '2': 'positive'}

**Load a csv dataset**

In [8]:
from datasets import load_dataset

dataset = load_dataset(
    "csv", 
    data_files = str((DATA_PATH / "train_data.csv").resolve(strict = True))
)

dataset

Using custom data configuration default-fb7c561e8af0be40


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /Users/huan/.cache/huggingface/datasets/csv/default-fb7c561e8af0be40/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 14640
    })
})

**Indexed by position**

In [9]:
for i in range(10):
    print(dataset['train'][i])

{'sentence': '@VirginAmerica What @dhepburn said.', 'label': 1}
{'sentence': "@VirginAmerica plus you've added commercials to the experience... tacky.", 'label': 2}
{'sentence': "@VirginAmerica I didn't today... Must mean I need to take another trip!", 'label': 1}
{'sentence': '@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse', 'label': 0}
{'sentence': "@VirginAmerica and it's a really big bad thing about it", 'label': 0}
{'sentence': "@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.\nit's really the only bad thing about flying VA", 'label': 0}
{'sentence': '@VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)', 'label': 2}
{'sentence': '@VirginAmerica Really missed a prime opportunity for Men Without Hats parody, there. https://t.co/mWpG7grEZP', 'label': 1}
{'sentence': "@virginamerica Well, I didn't…but NOW I DO! :-D", 'label': 2}
{'senten

**Indexed by column**

In [10]:
dataset['train']['sentence'][:3]

['@VirginAmerica What @dhepburn said.',
 "@VirginAmerica plus you've added commercials to the experience... tacky.",
 "@VirginAmerica I didn't today... Must mean I need to take another trip!"]

**Datasets are the same**     
1. Dataset Object. A container for data, central object of a dataset framework.
2. Dataset properties. Used to describe data.
3. Dataset transformations. Alter a dataset and returns a new dataset object.

In [11]:
dataset.sort('label')['train'][:10]['label']

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [12]:
dataset['train'][:10]['label']

[1, 2, 1, 0, 0, 0, 2, 1, 2, 2]

**trian test split**

In [13]:
split_dataset = dataset['train'].train_test_split(test_size = 0.3)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 10248
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 4392
    })
})

In [14]:
from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_fn(batch):
    return tokenizer(batch['sentence'], truncation = True)

tokenized_dataset = split_dataset.map(tokenize_fn, batched = True)


  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [15]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 3)

2023-01-10 23:10:50.548624: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that

In [16]:
training_args = TrainingArguments(
    output_dir = ARTIFACTS_PATH / "training_dir",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,    
)

In [17]:
import numpy as np
from sklearn.metrics import f1_score

def compute_metrics(logits_and_labels):
    logits, labels = logits_and_labels
    predictions = np.argmax(logits, axis = -1)
    return {"accuracy": np.mean(predictions == labels),"f1":f1_score(y_true = labels, y_pred = predictions, average = 'weighted')}

In [18]:
trainer = Trainer(
    model = model, 
    args = training_args, 
    train_dataset = tokenized_dataset['train'], 
    eval_dataset = tokenized_dataset['test'],
    tokenizer = tokenizer,
    compute_metrics=compute_metrics
    )

In [19]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/anaconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10248
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1923


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/1923 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 